In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,056 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [662 kB]
Get:13 http://archive.ubun

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [5]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"),sep=",", header=True)


In [6]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView('home_sales')


In [8]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
a = """
SELECT
  YEAR(date) AS YEAR,
  ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
WHERE bedrooms = 4
GROUP BY YEAR
ORDER BY YEAR DESC

"""

spark.sql(a).show()


+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2022|    296363.88|
|2021|    301819.44|
|2020|    298353.78|
|2019|     300263.7|
+----+-------------+



In [11]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
b = """
SELECT
  YEAR(date) AS YEAR,
  ROUND(AVG(price),2) AS AVERAGE_PRICE
FROM home_sales
WHERE bedrooms = 3
  AND bathrooms = 3
GROUP BY YEAR(date)
ORDER BY YEAR DESC
"""

spark.sql(b).show()



+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2022|    292725.69|
|2021|    294211.46|
|2020|    294204.16|
|2019|    287287.82|
+----+-------------+



In [14]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
c = """
SELECT
  YEAR (date_built) AS YEAR_BUILT,
  ROUND(AVG(price),2) AS AVERAGE_PRICE
FROM home_sales
WHERE bedrooms = 3
and bathrooms = 3
and sqft_living >= 2000
and floors = 2
GROUP BY YEAR_BUILT
ORDER BY YEAR_BUILT
"""
spark.sql(c).show()



+----------+-------------+
|YEAR_BUILT|AVERAGE_PRICE|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [15]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
d = """
SELECT
  view,
  ROUND(AVG(price),2) AS AVERAGE_PRICE
FROM home_sales
GROUP BY view
HAVING AVG(price) >=350000
ORDER BY view DESC
"""

spark.sql(d).show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 1.921997308731079 seconds ---


In [16]:
# 7. Cache the the temporary table home_sales.
spark.sql('cache table home_sales')

DataFrame[]

In [17]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [26]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

e = """

SELECT
  view,
  ROUND(AVG(price),2) AS AVERAGE_PRICE
FROM home_sales
GROUP BY view
HAVING AVG(price) >=35000
ORDER BY view desc
"""

spark.sql(e).show()
print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|   9|    282001.65|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.9999880790710449 seconds ---


In [28]:
print('the cached runtime of 0.99s was shorter than the uncached runtime of 1.92s')

the cached runtime of 0.99s was shorter than the uncached runtime of 1.92s


In [19]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.partitionBy("date_built").parquet('p_home_sales', mode = 'overwrite')

In [20]:
# 11. Read the parquet formatted data.
p_home_sales_df = spark.read.parquet('p_home_sales')

In [23]:
# 12. Create a temporary table for the parquet data.
p_home_sales_df.createOrReplaceTempView('p_temp_home_sales')

In [29]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()
f = """
SELECT
  view,
  ROUND(AVG(price),2) AS AVERAGE_PRICE
FROM p_temp_home_sales
GROUP BY view
HAVING AVG (price) >=350000
ORDER BY view DESC
"""

spark.sql(f).show()



print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 0.8656051158905029 seconds ---


In [31]:
print('the parquet runtime of 0.87s was shorter than the cached runtime of 0.99s')

the parquet runtime of 0.87s was shorter than the cached runtime of 0.99s


In [32]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table p_temp_home_sales')

DataFrame[]

In [33]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('p_temp_home_sales')


False